In [3]:
# Importando las bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
import SVM as svmf
import numpy as np

In [4]:

def loss(y, x, w, bias, c):
  hinge_loss = c * np.sum(np.maximum(0, 1 - y * (np.dot(x, w) + bias)))
  regularization_term = np.dot(w, w)/2
  return hinge_loss + regularization_term

def grad(y, x, w, bias, c):
  grad=[]
  if(y*(np.dot(x,w)+bias)>=1):
    grad.append(w)
    grad.append(0)
  else:
    grad.append(w - c*x*y)
    grad.append(-c*y)
  return grad

def update(w, b, grad, alpha):
    w -= alpha * grad[0]
    b -= alpha * grad[1]
    return w, b

class SVM:
    def __init__(self,C,epocas,alpha):
       self.C=C
       self.epocas=epocas 
       self.alpha=alpha

    def fit(self,x,y):
        self.clases = np.unique(y)
        self.clasicados = {}

        for class_label in self.clases:
            y_binario = np.where(y == class_label, 1, -1)
            clasificacion = self.train(x, y_binario)
            self.clasicados[class_label] = clasificacion

    def train(self,x,y):
        numero_ejemplos, num_caracteristicas = x.shape
        w = np.zeros(num_caracteristicas)
        bias = 0

        for e in range(self.epocas):
            for i in range(numero_ejemplos):
                condition = y[i] * (np.dot(x[i], w) - bias) >= 1
                if condition:
                    w -= self.alpha * (2 * self.C * w)
                else:
                    w -= self.alpha * (2 * self.C * w - np.dot(x[i], y[i]))
                    bias -= self.alpha * y[i]

        return (w, bias)
               
    def prediccion(self, x):
        predicciones = []

        for i in range(x.shape[0]):
            scores = {}
            for clase, classifier in self.clasicados.items():
                weights, bias = classifier
                score = np.dot(x[i], weights) - bias
                scores[clase] = score

            predicted_class = max(scores, key=scores.get)
            predicciones.append(predicted_class)

        return np.array(predicciones)   

In [5]:
def acuary(y_prueba,y_correct):
    correctos= np.sum(y_prueba == y_correct)
    return (correctos/len(y_correct))*100      

In [7]:
# Cargando el conjunto de datos de mariposas
data_mariposas = pd.read_csv('../Vectores_Caracteristicos_Mariposas.csv')

# Dividiendo el conjunto de datos en características (X) y etiquetas (y)
X_mariposas = data_mariposas.drop('Etiqueta', axis=1).values
y_mariposas = data_mariposas['Etiqueta'].values

# Dividiendo el conjunto de datos en conjuntos de entrenamiento y prueba
X_train_mariposas, X_test_mariposas, y_train_mariposas, y_test_mariposas = train_test_split(X_mariposas, y_mariposas, test_size=0.3, random_state=42)

modelo = SVM(1,300,0.00001)
modelo.fit(X_train_mariposas, y_train_mariposas)

y_pred_val = modelo.prediccion(X_test_mariposas)
precision_val=acuary(y_test_mariposas,y_pred_val)

print(f"Precisión en el conjunto de validación: {precision_val:.2f}")


Precisión en el conjunto de validación: 24.00
